In [1]:
# Importing the necessary libraries/modules
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Import the Axes3D module explicitly
from sklearn import neighbors
from astropy.table import Table

In [ ]:
#Vast libraries/modules
from vast.voidfinder.distance import z_to_comoving_dist
from vast.voidfinder import ra_dec_to_xyz
from vast.voidfinder._voidfinder_cython_find_next import MaskChecker

## Need to figure out how to get vast built in Jupyter.circ or sys.path.insert it in somehow
## Some path has changed and original insert not working

In [3]:
void_cat_A = ""
void_cat_B = ""

def calc_volume_boundaries(void_cat_A, void_cat_B):
    """Compute the boundaries of the minimal rectangular volume (parallelpiped)
    that completely contains two void catalogs.
   
    Parameters
    ----------
    void_cat_A : astropy.Table
        Table of void data from the first catalog.
    void_cat_B : astropy.Table
        Table of void data from the second catalog.
       
    Returns
    -------
    x_min : float
        Minimum x-coordinate of the enclosing volume.
    x_max : float
        Maximum x-coordinate of the enclosing volume.
    y_min : float
        Minimum y-coordinate of the enclosing volume.
    y_max : float
        Maximum y-coordinate of the enclosing volume.
    z_min : float
        Minimum z-coordinate of the enclosing volume.
    z_max : float
        Maximum z-coordinate of the enclosing volume.
    """
   
    # Calculating the minimum and maximum x-coordinates
    x_min = np.minimum(np.min(void_cat_A['x']), np.min(void_cat_B['x']))
    x_max = np.maximum(np.max(void_cat_A['x']), np.max(void_cat_B['x']))
   
    # ' ' y-coordinates
    y_min = np.minimum(np.min(void_cat_A['y']), np.min(void_cat_B['y']))
    y_max = np.maximum(np.max(void_cat_A['y']), np.max(void_cat_B['y']))

    # ' ' z-coordinates
    z_min = np.minimum(np.min(void_cat_A['z']), np.min(void_cat_B['z']))
    z_max = np.maximum(np.max(void_cat_A['z']), np.max(void_cat_B['z']))

    # Returning the boundaries
    return x_min, x_max, y_min, y_max, z_min, z_max

In [ ]:
calc_volume_boundaries(void_cat_A, void_cat_B)

def generate_grid_points(x_min, x_max, y_min, y_max, z_min, z_max):
    """Creates a dense rectangular grid of points in 3D for the void volume calculation.
   
    Parameters
    ----------
    x_min : float
        Minimum x-coordinate of the grid boundaries.
    x_max : float
        Maximum x-coordinate of the grid boundaries.
    y_min : float
        Minimum y-coordinate of the grid boundaries.
    y_max : float
        Maximum y-coordinate of the grid boundaries.
    z_min : float
        Minimum z-coordinate of the grid boundaries.
    z_max : float
        Maximum z-coordinate of the grid boundaries.
   
    Returns
    -------
    point_coords : numpy.ndarray with shape (3, N)
        The grid coordinates where N is the number of grid points.
    """
   
    # Defining the default grid spacing as 1 Megaparsec
    x_range = np.arange(x_min, x_max)
    y_range = np.arange(y_min, y_max)
    z_range = np.arange(z_min, z_max)

   
    # Creating a meshgrid from the input ranges
    X, Y, Z = np.meshgrid(x_range, y_range, z_range)

    x_points = np.ravel(X)
    y_points = np.ravel(Y)
    z_points = np.ravel(Z)
   
    point_coords = np.array([x_points, y_points, z_points])
   
    return point_coords

In [ ]:
## Need VAST to try to work on this func

def mask_point_filter(pts, mask, mask_resolution, rmin, rmax):
    """Determines which grid points generated by generate_grid_points lie in the survey mask
   
    Parameters
    ----------
    pts : numpy.ndarray with shape (3, N)
        The grid points generated by generate_grid_points.
    mask : numpy.ndarray with shape (N, M)
        The survey mask returned by vast.voidfinder.multizmask.generate_mask.
    mask_resolution : int
        The survey mask resolution returned by vast.voidfinder.multizmask.generate_mask.
    rmin : float
        The comoving minimum distance within the survey volume.
    rmax : float
        The comoving maximum distance within the survey volume.
   
    Returns
    -------
    points_in_mask : numpy.ndarray with shape (3, P)
        The list of grid points within the survey mask, where P is the number of points in the mask.
    """
    # Initializing a boolean array to tag points in the mask
    points_boolean = np.ones(pts.shape[1], dtype=bool)
   
    # Declaring a MaskChecker object from VAST
    mask_checker = MaskChecker(0,
                                mask,
                                mask_resolution,
                                rmin,
                                rmax)
   
    # Flagging the points that end up outside the mask
    for i in range(pts.shape[1]):
        # Get the current point
        curr_pt = pts[:, i]
        # Checking if the point is not in the mask
        not_in_mask = mask_checker.not_in_mask(curr_pt)
        # Invert not_in_mask to tag points in the mask
        points_boolean[i] = not bool(not_in_mask)
       
    # Now selecting all points that did not get filtered out during the above process
    points_in_mask = pts[:, points_boolean]
    return points_in_mask

In [5]:
def kd_tree(void_cat):
    """Creates a KD-Tree from the x-y-z coordinates of a void catalog.
   
    Parameters
    ----------
    void_cat : astropy.Table
        The given void catalog, which includes columns 'x', 'y', and 'z'.
   
    Returns
    -------
    sphere_tree : sklearn.neighbors._kd_tree.KDTree
        The KD-Tree object for the void catalog.
    """
    # Extracting x, y, and z coordinates from the void catalog
    cx = void_cat['x']
    cy = void_cat['y']
    cz = void_cat['z']

    # Formatting the void centers in an array of shape (N, 3)
    sphere_coords = np.array([cx, cy, cz]).T

    # Creating a KD-Tree from the coordinates using scikit-learn's neighbors module
    sphere_tree = neighbors.KDTree(sphere_coords)
   
    return sphere_tree

In [6]:
## Need VAST to try to work on this func

def point_query_VF(point_coords, sphere_tree, void_cat, max_sphere_size):
    """Determines which members of a set of grid points are inside VoidFinder voids
   
    Parameters
    ----------
    point_coords : numpy.ndarray with shape (3, N)
        The grid points within the survey volume.
    sphere_tree : sklearn.neighbors._kd_tree.KDTree
        The KD-Tree object for the void catalog generated by kd_tree.
    void_cat : astropy.Table
        The VoidFinder void catalog, which includes a void radius column 'radius'.
    max_sphere_size : float
        A currently unused parameter corresponding to the maximum sphere size in the void catalog.
   
    Returns
    -------
    true_inside : numpy.ndarray with shape (N,)
        Indicates which grid points are inside voids.
    """
    # Initialize an array that will tag which points are in voids
    true_inside = np.full(point_coords.shape[1], False, dtype=bool)
   
    # k determines the number of nearest neighbors in sphere_tree that are checked for each grid point
    # The default behavior is to check only the single nearest void for void membership.
    # It is possible that more distant voids are very large and enclose a grid point that is not enclosed by
    # its nearest neighbor void, but this source of error is currently not accounted for.
    # Setting k to a higher value will improve this error at the cost of runtime.
    k = min(sphere_tree.data.shape[0], 1)
   
    # Query the sphere tree
    dist, idx = sphere_tree.query(point_coords.T, k=k)
   
    # Checking which points are within void interiors
    interiors = dist < void_cat['radius'][idx]
   
    # Perform an OR operation on the k interiors columns to tag any point inside any of the k voids as belonging to a void
    true_inside += np.any(interiors, axis=1)
   
    return true_inside

In [7]:
## Need VAST to try to work on this func

def point_query_V2(point_coords, sphere_tree, void_cat):
    """Determines which members of a set of grid points are inside V2 voids
   
    Parameters
    ----------
    point_coords : numpy.ndarray with shape (3, N)
        The grid points within the survey volume.
    sphere_tree : sklearn.neighbors._kd_tree.KDTree
        The KD-Tree object for the void catalog generated by kd_tree.
    void_cat : astropy.Table
        The V2 void catalog which includes a void radius column 'radius'.
   
    Returns
    -------
    true_inside : numpy.ndarray with shape (N,)
        Indicates which grid points are inside voids.
    """
   
    idx = sphere_tree.query(point_coords.T, k=1, return_distance=False)
   
    true_inside = void_cat[idx]['in_void']

    return true_inside


In [8]:
def prep_V2_cat(V2_galzones, V2_zonevoids, data_table_vl):
    """Formates a V2 catalog for use with the void overlap calclator
   
    Parameters
    ----------
    V2_galzones: Astropy Table
        The V2 galzones catalogue
    V2_zonevoids: Astropy Table
        The V2 zonevoids catalogue
    data_table_vl: Astropy Table
        The galaxy catalog used to genreate the V2 void catalog
    Returns
    -------
    V2_galzones : Astropy table
        The V2_galzones object formatted for use with teh volume overlap caculator
    """
   
    # TODO: remove hardcoding of these values and allow user to set them
    omega_M = np.float32(0.3)
    h = np.float32(1.0)
   
    # Index the galaxies in the galaxy catalog
    if ~np.isin("index",data_table_vl.colnames):
        data_table_vl['index'] = np.arange(len(data_table_vl))
   
    # Sort the catalogs
    data_table_vl.sort('index')
    V2_galzones.sort('gal')
   
    # Ensure that each galaxy in the catalog has a zone inthe galzones file
    assert len(data_table_vl) == len(V2_galzones)

    # Modify the galaxy table column names
    # WARNING: don't input a catalog with x-y-z coordinates and no 'redshift' column
    # TODO: make this warning more clear to user
    if ~np.isin("redshift",data_table_vl.colnames):
        data_table_vl['z'].name = "redshift"
   
    # Calculate galaxy comoving distances
    Rgal = z_to_comoving_dist(data_table_vl['redshift'].astype(np.float32),omega_M,h)
    data_table_vl['Rgal'] = Rgal

    # Select only galaxies with positive redshift
    z_boolean = data_table_vl['redshift']>0
    data_table_vl = data_table_vl[z_boolean]
   
    # Calculate the xyz coordinates
    galaxies_xyz = ra_dec_to_xyz(data_table_vl)
    V2_galzones['x'] = galaxies_xyz[:,0]
    V2_galzones['y'] = galaxies_xyz[:,1]
    V2_galzones['z'] = galaxies_xyz[:,2]
   
    # Mark zones that belong to voids
    V2_galzones['in_void'] = V2_zonevoids[V2_galzones['zone']]['void1'] > -1
   
    return V2_galzones

In [9]:
def prep_V2_cat_NSA(V2_galzones, V2_zonevoids, data_table_vl):
    """Formats a V2 catalog for use with the void overlap calculator based on NSAID sorting
   
    Parameters
    ----------
    V2_galzones: Astropy Table
        The V2 galzones catalogue
    V2_zonevoids: Astropy Table
        The V2 zonevoids catalogue
    data_table_vl: Astropy Table
        The galaxy catalog used to generate the V2 void catalog
   
    Returns
    -------
    V2_galzones : Astropy Table
        The V2_galzones object formatted for use with the volume overlap calculator
    """
    #No sorting because the NSA catalogs are already sorted
   
    # Modify the galaxy table column names
    # WARNING: don't input a catalog with x-y-z coordinates and no 'redshift' column
    # TODO: make this warning more clear to user
    if ~np.isin("redshift",data_table_vl.colnames):
        data_table_vl['z'].name = "redshift"
   
    # Calculate galaxy comoving distances
    Rgal = z_to_comoving_dist(data_table_vl['redshift'].astype(np.float32),omega_M,h)
    data_table_vl['Rgal'] = Rgal

    # Select only galaxies with positive redshift
    z_boolean = data_table_vl['redshift']>0
    data_table_vl = data_table_vl[z_boolean]
   
    # Calculate the xyz coordinates
    galaxies_xyz = ra_dec_to_xyz(data_table_vl)
    V2_galzones['x'] = galaxies_xyz[:,0]
    V2_galzones['y'] = galaxies_xyz[:,1]
    V2_galzones['z'] = galaxies_xyz[:,2]
   
    # Mark zones that belong to voids
    V2_galzones['in_void'] = V2_zonevoids[V2_galzones['zone']]['void1'] > -1
   
    return V2_galzones


In [ ]:
# Loading V^2 Revolver void files
v2_1r_g = Table.read("V2_REVOLVER-nsa_v1_0_1_Planck2018_galzones.dat", format='ascii.commented_header')
v2_1r_z = Table.read("V2_REVOLVER-nsa_v1_0_1_Planck2018_zobovoids.dat", format='ascii.commented_header')

# Loading V^2 VIDE void files
v2_1v_g = Table.read("V2_VIDE-nsa_v1_0_1_Planck2018_galzones.dat", format='ascii.commented_header')
v2_1v_z = Table.read("V2_VIDE-nsa_v1_0_1_Planck2018_zobovoids.dat", format='ascii.commented_header')

# Loading galaxy table
galaxy_fits = Table.read("nsa_v1_0_1.fits", format='fits')

# Print column names of each table
print("Columns of V^2 Revolver Galzones:", v2_1r_g.colnames)
print("Columns of V^2 Revolver Zobovoids:", v2_1r_z.colnames)
print("Columns of V^2 VIDE Galzones:", v2_1v_g.colnames)
print("Columns of V^2 VIDE Zobovoids:", v2_1v_z.colnames)
print("Columns of Galaxy Table:", galaxy_fits.colnames)